In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

import numpy as np
from numpy import random
from numpy import argmax
from numpy import asarray, zeros
import pickle

In [2]:
NUM_SENTENCES = 10000 # Use only the first 20,000 records.
MAX_NUM_WORDS = 10000 # Use 20,000 words for tokenizing
MAX_SENT_LEN = 50

EMBEDDING_SIZE = 100

LSTM_NEURONS = 100

BATCH_SIZE = 64
EPOCHS = 5

In [3]:
inputs = []
outputs_i = []
outputs = []

# Define the file paths
input_file_path = 'english.txt'  # Source sentences
output_file_path = 'indonesian.txt'  # Target sentences

# Read input sentences
with open(input_file_path, 'r', encoding='utf-8') as input_file:
    for line in input_file:
        inputs.append(line.strip())

# Read target sentences
with open(output_file_path, 'r', encoding='utf-8') as output_file:
    for line in output_file:
        # Add special tokens for intermediate and final outputs
        temp_op = line.strip()
        op_i = '<sos> ' + temp_op  # Add start-of-sequence token
        op = temp_op + ' <eos>'   # Add end-of-sequence token
        outputs_i.append(op_i)
        outputs.append(op)

#Take Max
max=10000
inputs=inputs[:max]
outputs_i=outputs_i[:max]
outputs=outputs[:max]


# Print statistics and a sample
print('Total inputs =', len(inputs))
print('Total intermediate outputs =', len(outputs_i))
print('Total outputs =', len(outputs))
print('\nSample:')
print(inputs[100])     # Example input sentence
print(outputs_i[100])  # Intermediate output with <sos>
print(outputs[100])    # Final output with <eos>

Total inputs = 10000
Total intermediate outputs = 10000
Total outputs = 10000

Sample:
Welcome.
<sos> Selamat datang.
Selamat datang. <eos>


In [4]:
from builtins import max
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(inputs)

inputs_seq = input_tokenizer.texts_to_sequences(inputs)

inputs_word2index = input_tokenizer.word_index
print('Total unique words in input:', len(inputs_word2index))

inputs_numwords = len(inputs_word2index)+1

inputs_maxlen = max(len(s) for s in inputs_seq)
print('Length of longest sentence in input:', inputs_maxlen)

output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(outputs_i + outputs)

outputs_i_seq = output_tokenizer.texts_to_sequences(outputs_i)
outputs_seq = output_tokenizer.texts_to_sequences(outputs)

outputs_word2index = output_tokenizer.word_index
print('Total unique words in output:', len(outputs_word2index))

outputs_numwords = len(outputs_word2index)+1

outputs_maxlen = max(len(s) for s in outputs_seq)
print('Length of longest sentence in output:', outputs_maxlen)

Total unique words in input: 3615
Length of longest sentence in input: 9
Total unique words in output: 6093
Length of longest sentence in output: 16


In [5]:
encoder_input_sequences = pad_sequences(inputs_seq, maxlen=inputs_maxlen)
print('encoder_input_sequences shape:', encoder_input_sequences.shape)

decoder_input_sequences = pad_sequences(outputs_i_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_inputs_sequences shape:', decoder_input_sequences.shape)

decoder_output_sequences = pad_sequences(outputs_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_output_sequences shape:', decoder_output_sequences.shape)

encoder_input_sequences shape: (10000, 9)
decoder_inputs_sequences shape: (10000, 16)
decoder_output_sequences shape: (10000, 16)


In [6]:
embeddings_dict = dict()

glove_file = open('glove.6B.100d.txt', encoding='utf8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector = asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector

glove_file.close()

num_words = min(MAX_NUM_WORDS, len(inputs_word2index)+1)

embedding_matrix = zeros((num_words, EMBEDDING_SIZE))

for word, index in inputs_word2index.items():
    if index < num_words:  # Ensure the index is within bounds
        embedding_vector = embeddings_dict.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector


# Model

In [7]:
# Encoder embedding
encoder_embedding_layer = Embedding(
    num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=inputs_maxlen
)

# Decoder embedding
decoder_embedding_layer = Embedding(outputs_numwords, EMBEDDING_SIZE)

# Encoder model
encoder_inputs = Input(shape=(inputs_maxlen,))
encoder_inputs_emb = encoder_embedding_layer(encoder_inputs)
encoder = LSTM(LSTM_NEURONS, return_state=True)
encoder_outputs, h, c = encoder(encoder_inputs_emb)
encoder_states = [h, c]

# Decoder model
decoder_inputs = Input(shape=(outputs_maxlen,))
decoder_inputs_emb = decoder_embedding_layer(decoder_inputs)
decoder = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs_emb, initial_state=encoder_states)

# Dense layer for output
output_dense_layer = Dense(outputs_numwords, activation="softmax")
outputs = output_dense_layer(decoder_outputs)

# Final model
model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 9)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 16)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 9, 100)         │        361,600 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 16, 100)        │        609,400 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 100), (None,   │         80,400 │ embedding[0][0]        │
│                           │ 100), (None, 100)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 16, 100),      │         80,400 │ embedding_1[0][0],     │
│                           │ (None, 100), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 100)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 16, 6094)       │        615,494 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,747,294 (6.67 MB)

 Trainable params: 1,747,294 (6.67 MB)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
decoder_outputs_onehot = to_categorical(decoder_output_sequences, num_classes=outputs_numwords)

# Fit the model with preprocessed one-hot encoded outputs
trn = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_outputs_onehot,
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 56ms/step - accuracy: 0.6668 - loss: 4.9568 - val_accuracy: 0.6306 - val_loss: 2.6253
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - accuracy: 0.7611 - loss: 1.7674 - val_accuracy: 0.6737 - val_loss: 2.4275
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.7739 - loss: 1.6372 - val_accuracy: 0.6833 - val_loss: 2.3804
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.7796 - loss: 1.5951 - val_accuracy: 0.6853 - val_loss: 2.3494
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.7828 - loss: 1.5504 - val_accuracy: 0.6899 - val_loss: 2.3127
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.7874 - loss: 1.5209 - val_accuracy: 0.6915 - val_loss: 2.2998
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7889 - loss: 1.4906 - val_accuracy: 0.6933 - val_loss: 2.2647
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.7922 - loss: 1.4642 - val_a

In [9]:
encoder_model = Model(encoder_inputs, encoder_states)
print(encoder_model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 9)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 9, 100)              │         361,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ [(None, 100), (None, 100),  │          80,400 │
│                                      │ (None, 100)]                │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 442,000 (1.69 MB)

 Trainable params: 442,000 (1.69 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
decoder_input_h = Input(shape=(LSTM_NEURONS,))
decoder_input_c = Input(shape=(LSTM_NEURONS,))
decoder_input_states = [decoder_input_h, decoder_input_c]

decoder_input_word = Input(shape=(1,))
decoder_input_word_emb = decoder_embedding_layer(decoder_input_word)

decoder_outputs, h, c = decoder(decoder_input_word_emb, initial_state=decoder_input_states)
decoder_states = [h, c]

outputs = output_dense_layer(decoder_outputs)

In [11]:
decoder_model = Model([decoder_input_word]+decoder_input_states, [outputs]+decoder_states)
print(decoder_model.summary())

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 100)         │        609,400 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 1, 100),       │         80,400 │ embedding_1[1][0],     │
│                           │ (None, 100), (None,    │                │ input_layer_2[0][0],   │
│                           │ 100)]                  │                │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1, 6094)        │        615,494 │ lstm_1[1][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,305,294 (4.98 MB)

 Trainable params: 1,305,294 (4.98 MB)

 Non-trainable params: 0 (0.00 B)

None


In [14]:
index_to_word_input = {v:k for k,v in inputs_word2index.items()}
index_to_word_output = {v:k for k,v in outputs_word2index.items()}
# Preprocess input text
def preprocess_input(text):
    # Use 0 as the default index for unknown words
    tokens = [inputs_word2index.get(word, 0) for word in text.lower().split()]
    padded_seq = pad_sequences([tokens], maxlen=inputs_maxlen, padding='post')
    return padded_seq

# Translate function
def translate(txt):
    input_seq = preprocess_input(txt)
    states = encoder_model.predict(input_seq)

    sos = outputs_word2index['<sos>']
    eos = outputs_word2index['<eos>']

    output_seq = np.zeros((1, 1))
    output_seq[0, 0] = sos

    output_sentence = []

    for _ in range(outputs_maxlen):
        output_tokens, h, c = decoder_model.predict([output_seq] + states)
        idx = np.argmax(output_tokens[0, 0, :])

        if idx == eos:
            break
        if idx > 0:
            word = index_to_word_output[idx]
            output_sentence.append(word)

        states = [h, c]
        output_seq[0, 0] = idx

    return ' '.join(output_sentence)

In [18]:
# Example usage
text = "We have to keep the car go"
translation = translate(text)

print('Input:', text)
print('Translation:', translation)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Input: We have to keep the car go
Translation: kita harus membeli dua mobil.


In [19]:
encoder_model.save("encoder_model.h5")
decoder_model.save("decoder_model.h5")